In [1]:
!pip list | grep jax
!pip list | grep flax
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import time
import requests
import functools
import jax
from jax import config
import jax.numpy as jnp
import flax
from matplotlib import pyplot as plt
import numpy as onp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import diffusion_distillation
from flax.training import checkpoints as flaxcheckpoints



jax                          0.3.15
jaxlib                       0.3.15+cuda11.cudnn82
flax                         0.4.2


2024-04-18 11:07:19.524543: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 11:07:20.307471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.1/lib64:/usr/local/cuda-12.1/lib64:
2024-04-18 11:07:20.307731: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.1/lib64:/usr/local/cuda-12.1/lib64:
2024-04-18 11:07:20.307751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with Tensor

## Train a new diffusion model

In [6]:
# jax.devices()
# jax.config.FLAGS.jax_xla_backend = "gpu_driver"
print("JAX backend:", jax.lib.xla_bridge.get_backend().platform)
jax.devices()


JAX backend: gpu


[GpuDevice(id=0, process_index=0)]

In [7]:
# create model
config = diffusion_distillation.config.cifar_base.get_config()
model = diffusion_distillation.model.Model(config)

In [8]:
print(type(model.make_optimizer_def()))
# print(type(model.make_optimizer_def().init(None)))

<class 'flax.optim.adam.Adam'>


/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(


In [9]:
# init params 
state = jax.device_get(model.make_init_state())
state = flax.jax_utils.replicate(state)

/home/ljb/new_diffusion_distillation/diffusion_distillation/model.py:96: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  jax.tree_map(lambda a: a.shape, init_params)))
/home/ljb/new_diffusion_distillation/diffusion_distillation/utils.py:290: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  return sum([x.size for x in jax.tree_leaves(pytree)])
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:103: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  param_states = jax.tree

In [10]:
type(model.make_optimizer_def())

/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(


flax.optim.adam.Adam

In [11]:
# type(state.optimizer[0]), type(state.optimizer[1])

In [12]:
model.make_init_state()

/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(


TrainState(step=0, optimizer=Optimizer(optimizer_def=<flax.optim.adam.Adam object at 0x79640c604a60>, state=OptimizerState(step=DeviceArray(0, dtype=int32), param_states=FrozenDict({
    conv_in: {
        bias: _AdamParamState(grad_ema=DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0., 0., 0., 0

In [13]:
# JIT compile training step
train_step = functools.partial(model.step_fn, jax.random.PRNGKey(0), True)
train_step = functools.partial(jax.lax.scan, train_step)  # for substeps
train_step = jax.pmap(train_step, axis_name='batch', donate_argnums=(0,))

In [14]:
# build input pipeline
total_bs = config.train.batch_size
device_bs = total_bs // jax.device_count()
train_ds = model.dataset.get_shuffled_repeated_dataset(
    split='train',
    batch_shape=(
        jax.local_device_count(),  # for pmap
        config.train.substeps,  # for lax.scan over multiple substeps
        device_bs,  # batch size per device
    ),
    local_rng=jax.random.PRNGKey(0),
    augment=True)
train_iter = diffusion_distillation.utils.numpy_iter(train_ds)

In [17]:
# run training
for step in range(1):
  batch = next(train_iter)
  state, metrics = train_step(state, batch)
  print(flaxcheckpoints.save_checkpoint('/tmp/flax_ckpt/checkpoint/', state, step, overwrite=True))
  metrics = jax.device_get(flax.jax_utils.unreplicate(metrics))
  metrics = jax.tree_map(lambda x: float(x.mean(axis=0)), metrics)
  print(metrics)

{'train/gnorm': 710.9559326171875, 'train/loss': 8.86741828918457}


/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/jax_utils.py:61: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  return jax.tree_map(lambda x: x[0], tree)
/tmp/ipykernel_2408007/3268855046.py:7: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  metrics = jax.tree_map(lambda x: float(x.mean(axis=0)), metrics)


In [ ]:
# get all attr
#print(dir(state))
#for k, v in dict(state.ema_params).items():
#    print(k, type(v))
#print()
print(state.num_sample_steps)
print(model.make_init_state().num_sample_steps)
#print(state.optimizer)
#print(state.replace)
#print(state.step)

[0]


/home/ljb/new_diffusion_distillation/diffusion_distillation/utils.py:290: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  return sum([x.size for x in jax.tree_leaves(pytree)])
/home/ljb/miniforge3/envs/diffd/lib/python3.10/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(


0


## Distill a trained diffusion model

In [2]:
# create model
import diffusion_distillation
config = diffusion_distillation.config.cifar_distill.get_config()
model = diffusion_distillation.model.Model(config)

In [3]:
# from flax import serialization

# def restore_from_path(ckpt_path, target):
#   with open(ckpt_path, 'rb') as fp:
#     return serialization.from_bytes(target, fp.read())
# loaded = flaxcheckpoints.restore_checkpoint("/tmp/flax_ckpt/checkpoint/checkpoint_0", None)
# print(loaded)

In [4]:
# load the teacher params
model.load_teacher_state("/tmp/flax_ckpt/checkpoint/checkpoint_0")

/home/ljb/new_diffusion_distillation/diffusion_distillation/model.py:97: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  jax.tree_map(lambda a: a.shape, init_params)))
/home/ljb/new_diffusion_distillation/diffusion_distillation/utils.py:290: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  return sum([x.size for x in jax.tree_leaves(pytree)])
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:103: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  param_states = jax.tree

In [5]:
# init student state
init_params = diffusion_distillation.utils.copy_pytree(model.teacher_state.ema_params)
optim = model.make_optimizer_def().create(init_params)
state = diffusion_distillation.model.TrainState(
    step=model.teacher_state.step,
    optimizer=optim,
    ema_params=diffusion_distillation.utils.copy_pytree(init_params),
    num_sample_steps=model.teacher_state.num_sample_steps//2)

/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/optim/base.py:49: DeprecationWarning: Use `optax` instead of `flax.optim`. Refer to the update guide https://flax.readthedocs.io/en/latest/howtos/optax_update_guide.html for detailed instructions.
  warnings.warn(


In [6]:
# build input pipeline
total_bs = config.train.batch_size
device_bs = total_bs // jax.device_count()
train_ds = model.dataset.get_shuffled_repeated_dataset(
    split='train',
    batch_shape=(
        jax.local_device_count(),  # for pmap
        config.train.substeps,  # for lax.scan over multiple substeps
        device_bs,  # batch size per device
    ),
    local_rng=jax.random.PRNGKey(0),
    augment=True)
train_iter = diffusion_distillation.utils.numpy_iter(train_ds)

In [7]:
steps_per_distill_iter = 10  # number of distillation steps per iteration of progressive distillation
end_num_steps = 4  # eventual number of sampling steps we want to use 
while state.num_sample_steps >= end_num_steps:

  # compile training step
  train_step = functools.partial(model.step_fn, jax.random.PRNGKey(0), True)
  train_step = functools.partial(jax.lax.scan, train_step)  # for substeps
  train_step = jax.pmap(train_step, axis_name='batch', donate_argnums=(0,))

  # train the student against the teacher model
  print('distilling teacher using %d sampling steps into student using %d steps'
        % (model.teacher_state.num_sample_steps, state.num_sample_steps))
  state = flax.jax_utils.replicate(state)
  for step in range(steps_per_distill_iter):
    batch = next(train_iter)
    state, metrics = train_step(state, batch)
    metrics = jax.device_get(flax.jax_utils.unreplicate(metrics))
    metrics = jax.tree_map(lambda x: float(x.mean(axis=0)), metrics)
    print(metrics)

  # student becomes new teacher for next distillation iteration
  model.teacher_state = jax.device_get(
      flax.jax_utils.unreplicate(state).replace(optimizer=None))

  # reset student optimizer for next distillation iteration
  init_params = diffusion_distillation.utils.copy_pytree(model.teacher_state.ema_params)
  optim = model.make_optimizer_def().create(init_params)
  state = diffusion_distillation.model.TrainState(
      step=model.teacher_state.step,
      optimizer=optim,
      ema_params=diffusion_distillation.utils.copy_pytree(init_params),
      num_sample_steps=model.teacher_state.num_sample_steps//2)

distilling teacher using 8192 sampling steps into student using 4096 steps


/home/ljb/new_diffusion_distillation/diffusion_distillation/utils.py:198: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  return jax.tree_map(lambda x: (x._numpy()
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/ljb/miniforge3/envs/dd/lib/python3.8/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  value_flat = jax.tree_leaves(value)
/home/ljb/new_diffusion_distillation/diffusion_distillation/utils.py:299: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  [jnp.sum(jnp.square(x)) for x in jax.tree_leaves(pytree)])))
/